# Comparison initial / regridded files for all the models 

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob

In [3]:
# models
models = [
    "BCC-CSM2-MR", "CanESM5", "CanESM5-CanOE", "CESM2", "CESM2-WACCM", "CIESM",
    "CNRM-CM6-1", "CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-Veg",
    "FGOALS-f3-L", "FGOALS-g3", "GFDL-CM4", "GISS-E2-1-G", "HadGEM3-GC31-LL",
    "IPSL-CM6A-LR", "MIROC-ES2L", "MIROC6", "MPI-ESM1-2-HR", "MPI-ESM1-2-LR",
    "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", "UKESM1-0-LL"
]

In [4]:
# path initial files
path_ini_hist = "/home/jovyan/private-storage/historical-LImon-snc/"

In [5]:
datasets_ini_hist = {}

for model in models:
    file_pattern = os.path.join(path_ini_hist, f"snc_LImon_{model}_historical_*.nc")
    files = glob.glob(file_pattern)
    
    datasets_ini_hist[model] = xr.open_mfdataset(files, combine='by_coords')

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 'snc' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 'snc' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [7]:
# areacella
path_areacella = "/home/jovyan/private-storage/masques/masques/areacella/"
path_pseudo_areacella = "/home/jovyan/private-storage/masques/masques/pseudo_areacella/"

In [8]:
datasets_area = {}
for model in models:
    area_file_path = os.path.join(path_areacella, f"areacella_{model}.nc")
    pseudo_area_file_path = os.path.join(path_pseudo_areacella, f"areacella_{model}.nc")

    if os.path.exists(area_file_path):
        datasets_area[model] = xr.open_dataset(area_file_path)['areacella']
    elif os.path.exists(pseudo_area_file_path):
        datasets_area[model] = xr.open_dataset(pseudo_area_file_path)['areacella']
    else:
        print(f"Pas de fichier d'aire trouvé pour {model} !")
        datasets_area[model] = None

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 'areacella' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [ ]:
path_BCC_CSM2 = os.path.join("/home/jovyan/private-storage/historical-LImon-snc/", "snc_LImon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.nc")

In [ ]:
BCC_CSM2 = xr.open_dataset(path_BCC_CSM2)
BCC_CSM2

In [ ]:
snow_cover_ini = BCC_CSM2["snc"]
snow_cover_ini

In [ ]:
path_areacella_BCC_CSM2 = os.path.join("/home/jovyan/private-storage/masques/masques/pseudo_areacella/", "areacella_BCC-CSM2-MR.nc")

In [ ]:
areacella_BCC_CSM2 = xr.open_dataset(path_areacella_BCC_CSM2)
areacella_BCC_CSM2

In [ ]:
BCC_CSM2_ref = BCC_CSM2.sel(time=slice("1995", "2014"))
BCC_CSM2_ref

In [ ]:
snow_cover = BCC_CSM2_ref["snc"]

In [ ]:
snow_cover = snow_cover.groupby("time.month").mean(dim="time")

In [ ]:
def plot(data, x, y, title):
    plt.plot(range(1, 13), data, marker="o")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(title)
    plt.grid()
    plt.show()

In [ ]:
monthly_mean = snow_cover.mean(dim=("lat", "lon"))

plot(
    data=monthly_mean,
    x="Mois",
    y="Couverture neigeuse moyenne",
    title="Moyenne mensuelle de la couverture neigeuse"
)

In [ ]:
cell_area_ini = areacella_BCC_CSM2["areacella"]

snow_cover_km2 = (snow_cover * (cell_area_ini * 1e-6) ).groupby(snow_cover["month"]).sum(dim=["lat", "lon"], skipna=True)

print(snow_cover_km2)
print(snow_cover_km2.shape)

In [ ]:
plot(
    data=snow_cover_km2,
    x="Mois",
    y="Superficie enneigée (km²)",
    title="Superficie moyenne mensuelle de la couverture neigeuse"
)